In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib


In [19]:
# 1. 加载并合并数据
years = [2015, 2016, 2017, 2018, 2019]
data_frames = [pd.read_excel(os.path.join('Dataset', f'{year}.xlsx')) for year in years]
data = pd.concat(data_frames, ignore_index=True)


In [20]:
print(data.shape)

(72370, 192)


In [21]:
# 2. 定义特征类型
categorical_cols = ['Nationality', 'Country of residence', 'Gender', 'Immigration airport', 'Travel type', 
                    'Having mileage or not', 'Most desired place', 'Most satisfied place', 'Satisfaction level', 
                    'Intention to revisit CITY']
numerical_cols = ['Age', 'Number of visits to CITY', 'Tour price', 'Airfare', 'Total expenditures', 
                  'Accommodation expenses', 'Food and drink expenses', 'Transportation expenses in CITY', 
                  'Entertainment expenses', 'Shopping expenses', 'Other expenditures'] + \
                 [f'Satisfaction level by item {i:02d}' for i in range(1, 12)]
binary_cols = [f'Companion_{i}' for i in range(1, 7)] + \
              [f'Accommodation facilities {i}' for i in range(1, 9)] + \
              [f'Transportation {i:02d}' for i in range(1, 18)] + \
              [f'Activity {i:02d}' for i in range(1, 23)] + \
              [f'Destination {i:02d}' for i in range(1, 25)] + \
              [f'Activity_Destination {i:02d}' for i in range(1, 25)] + \
              [f'Trigger {i:02d}' for i in range(1, 30)] + \
              [f'Attractions {i:02d}' for i in range(1, 28)]

In [22]:
# 3. 转换为日期格式并按时间排序
data['Survey date'] = pd.to_datetime(data['Survey date'], errors='coerce')
data = data.sort_values('Survey date').reset_index(drop=True)

In [23]:
# 4. 预处理：处理缺失值
data[categorical_cols] = SimpleImputer(strategy='most_frequent').fit_transform(data[categorical_cols])
data[numerical_cols] = SimpleImputer(strategy='mean').fit_transform(data[numerical_cols])
data[binary_cols] = SimpleImputer(strategy='most_frequent').fit_transform(data[binary_cols])

In [24]:
# 5. 对类别特征进行独热编码
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [25]:
# 6. 回归训练数据构建
reg_data = data.dropna(subset=['Number of nights in CITY'])
X_reg = reg_data.drop(['Survey date', 'Number of nights in CITY', 'Purpose of visit to CITY'], axis=1)
y_reg = reg_data['Number of nights in CITY']

# 7. 分类训练数据构建
cls_data = data.dropna(subset=['Purpose of visit to CITY'])
X_cls = cls_data.drop(['Survey date', 'Number of nights in CITY', 'Purpose of visit to CITY'], axis=1)
y_cls = cls_data['Purpose of visit to CITY']


In [26]:
#  保存
os.makedirs('initial_data', exist_ok=True)
np.savez('initial_data/initial_data_rolling.npz',
         X_reg_ini=X_reg.to_numpy(dtype=np.float32),
         y_reg_ini=y_reg.values,
         X_cls_ini=X_cls.to_numpy(dtype=np.float32),
         y_cls_ini=y_cls.values)

In [27]:
# 8. 标准化
scaler = StandardScaler()
X_reg_scaled = scaler.fit_transform(X_reg)
X_cls_scaled = scaler.transform(X_cls)  # 使用相同的scaler

In [28]:
# 9. 保存
os.makedirs('processed_data', exist_ok=True)
np.savez('processed_data/processed_data_rolling.npz',
         X_reg_scaled=X_reg_scaled,
         y_reg=y_reg.values,
         X_cls_scaled=X_cls_scaled,
         y_cls=y_cls.values)
joblib.dump(scaler, 'processed_data/scaler.pkl')

['processed_data/scaler.pkl']

In [29]:
print("回归数据:", X_reg_scaled.shape)
print("分类数据:", X_cls_scaled.shape)

回归数据: (72054, 379)
分类数据: (72370, 379)
